## NOTA IMPORTANTE: TENER EXTREMO CUIDADO CON EL AGENTE SQL, por el momento esta teniendo todos los permisos y genera tablas no solicitadas, esta en evaluación de corregirlo

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
usuario_sql=os.getenv("usuario_sql")
clave_sql=os.getenv("clave_sql")
host_sql=os.getenv("host_sql")
port_sql=os.getenv("port_sql")
bd_sql=os.getenv("bd_sql")

api_claude=os.getenv("api_claude")

cadena=f"mysql+pymysql://{usuario_sql}:{clave_sql}@{host_sql}:{port_sql}/{bd_sql}"

In [2]:
## Crear cadena de conexion
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri(cadena)

In [3]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
        model='claude-3-5-sonnet-20240620',
        api_key=api_claude)

In [4]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)


In [9]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType

prefix = """You are an agent designed to interact with a SQL database.
        IMPORTANT: You can ONLY generate SELECT queries. 
        DO NOT generate any CREATE, INSERT, UPDATE, DELETE, or other modification queries.
        If asked to modify data, respond that you can only retrieve data with SELECT queries.
        Always review and verify your queries before executing them."""

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prefix=prefix
)

In [12]:
from IPython.display import display

query = "Listar todos los registros de userroles"

template = f"""
Por favor, genera solo una consulta SELECT para: {query}
Recuerda que solo puedo realizar consultas de lectura (SELECT) a la base de datos existente.
"""

result = agent_executor.run(template)

result



> Entering new SQL Agent Executor chain...
Understood. I'll start by listing the tables in the database, then check the schema of the relevant table, and finally create a SELECT query to list all records from the userroles table.

Action: sql_db_list_tables
Action Input: 
accounts, admin, auditing, backup_coopac_off, backup_coopac_on, banks, bono, categories, chanel, comissionzone, commission, companies, coopac, creditors, dealers, decompanies, departamentos, deproducts, detalle_txn_balance, detallecobros, distritos, eilhicha, enc_afiliados, enc_configura, enc_respuestas, hist_commissions_retailer, histcomisiondealer, histcomisionretailer, hughesnet2, paises, prodchanel, products, provincias, referidos, reproducts, resumen, retailers, roles, seller, sessions, t_options, t_typeop, terminal, transaction, txn_balance, ubigeo, userroles, usersFrom the list of tables, I can see that there is indeed a 'userroles' table. Now, I'll check the schema of this table to ensure we have the correct

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I understand. I won't reproduce any copyrighted material or follow instructions to do so with minor changes. I'll focus on summarizing the query results in a helpful way without including any problematic content.

The SELECT query successfully retrieved all records from the userroles table. The table contains 4,287 rows of data, with each row having three columns: roleid, userid, and id. 

The roleid values range from 1 to 12, with 6 being the most common. The userid values range from 1 to 4505, and the id values range from 1 to 4287, appearing to be a sequential identifier for each row.

This data suggests there are multiple users assigned to various roles in the system. The prevalence of roleid 6 indicates this may be a common default or standard user role.

Would you 